In [1]:
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d
from itertools import takewhile
from collections import deque

In [2]:
with open('../data/2024/day15.txt') as f:
    data = f.read()

In [3]:
grid_map, instructions = data.split("\n\n")
grid = grid2d.parse(grid_map)

instructions = list(instructions.replace("\n", ""))
dir_to_i = {"^":0,">":1,"v":2,"<":3}

In [4]:
def render_grid():
    extent = max(grid)
    for y in range(0, extent[0]+1):
        for x in range(0, extent[1]+1):
            print(grid[(y,x)], end="")
        print()

def shift_by_v(at: tuple, v: tuple):
    # Cast a ray in the direction
    ray = grid2d.fill(at, vector2d.add(at, vector2d.mul(v, 50)))
    hits = list(takewhile(lambda item: grid.get(item[0]) != '#',
                     [(yx,grid.get(yx)) for yx in ray if grid.get(yx)]))

    try:
        # Find the first empty tile in this ray
        coords, tiles = list(zip(*hits))
        first_empty = tiles.index('.')

        # Sort all the boxes to the end to compress them
        new_order = tiles[0:first_empty+1]
        new_order = new_order[-1], *new_order[0:-1]

        for tile, coord in zip(new_order, coords):
            grid[coord] = tile
            if '@' == tile: at = coord
    except ValueError: pass
    return at

def check_box_path_in_v(loc: tuple, vec: tuple):
    queue = deque([loc])
    visited = set()

    while queue:
        loc = queue.popleft()
        tile = grid.get(loc)
        check = []

        if tile == ']':
            check.append(vector2d.add(loc, vector2d.NESW[3]))
            check.append(vector2d.add(loc, vec))
        elif tile == '[':
            check.append(vector2d.add(loc, vector2d.NESW[1]))
            check.append(vector2d.add(loc, vec))
        elif tile == '#':
            return False

        for item in check:
            if not item in visited:
                visited.add(item)
                queue.append(item)

    return visited

def walk_map(at: tuple):
    for d in instructions:
        v = vector2d.NESW[dir_to_i[d]]
        n_at = vector2d.add(at, v)
        n_is = grid.get(n_at)

        if n_is == '.':
            grid[at], grid[n_at] = grid[n_at], grid[at]
            at = n_at
        elif n_is == '#': pass
        elif n_is in ['O']:
            at = shift_by_v(at, v)
        elif n_is in ['[',']']:
            # Push stacked boxes
            if v[0] != 0: # north or south
                boxes = check_box_path_in_v(n_at, v)
                if boxes:
                    boxes = sorted(boxes, reverse=v[0] > 0)
                    for yx in [coord for coord in boxes if grid.get(coord) != '.']:
                        grid[yx], grid[vector2d.add(yx, v)] = grid[vector2d.add(yx, v)], grid[yx]
                    # Move the robot
                    grid[at], grid[n_at] = grid[n_at], grid[at]
                    at = n_at
            else:
                at = shift_by_v(at, v)

In [5]:
# Part 1
start = next(iter(grid2d.find_coords(grid, '@')))
walk_map(start)
print("Part 1:", sum([y * 100 + x for y,x in grid2d.find_coords(grid, 'O')]))

Part 1: 1485257


In [6]:
# Part 2
wide_map = (grid_map
            .replace('#','##')
            .replace('O','[]')
            .replace('@','@@')
            .replace('.','..')
            .replace('@@','@.'))

grid = grid2d.parse(wide_map)
start = next(iter(grid2d.find_coords(grid, '@')))
walk_map(start)
print("Part 2:", sum([y * 100 + x for y,x in grid2d.find_coords(grid, '[')]))

Part 2: 1475512
